# Softmax exercise

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

This exercise is analogous to the SVM exercise. You will:

- implement a fully-vectorized **loss function** for the Softmax classifier
- implement the fully-vectorized expression for its **analytic gradient**
- **check your implementation** with numerical gradient
- use a validation set to **tune the learning rate and regularization** strength
- **optimize** the loss function with **SGD**
- **visualize** the final learned weights


In [1]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

from __future__ import print_function

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]
    
    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
    
    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image
    
    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
    
    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
try:
   del X_train, y_train
   del X_test, y_test
   print('Clear previously loaded data.')
except:
   pass

# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


## Softmax Classifier

Your code for this section will all be written inside **cs231n/classifiers/softmax.py**. 


In [3]:
# First implement the naive softmax loss function with nested loops.
# Open the file cs231n/classifiers/softmax.py and implement the
# softmax_loss_naive function.

from cs231n.classifiers.softmax import softmax_loss_naive
import time

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)
print('sanity check: %f' % (-np.log(0.1)))

loss: 2.421511
sanity check: 2.302585


## Inline Question 1:
Why do we expect our loss to be close to -log(0.1)? Explain briefly.**

**Your answer:** *Fill this in*


In [4]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As we did for the SVM, use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from cs231n.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# similar to SVM case, do another gradient check with regularization
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

numerical: 0.722560 analytic: 0.722560, relative error: 3.763249e-08
numerical: 2.180948 analytic: 2.180948, relative error: 2.128063e-08
numerical: -0.481055 analytic: -0.481055, relative error: 8.331068e-08
numerical: -2.922778 analytic: -2.922778, relative error: 7.622307e-09
numerical: -1.229095 analytic: -1.229095, relative error: 3.972646e-09
numerical: 1.890958 analytic: 1.890958, relative error: 2.519403e-08
numerical: -0.878980 analytic: -0.878980, relative error: 1.398525e-08
numerical: -1.690864 analytic: -1.690864, relative error: 5.242837e-10
numerical: 1.287436 analytic: 1.287436, relative error: 4.623734e-08
numerical: 1.241083 analytic: 1.241083, relative error: 4.876583e-08
numerical: 2.305517 analytic: 2.295596, relative error: 2.156208e-03
numerical: 0.804522 analytic: 0.815819, relative error: 6.971386e-03
numerical: -3.880797 analytic: -3.891943, relative error: 1.433940e-03
numerical: 2.448035 analytic: 2.444576, relative error: 7.069611e-04
numerical: 2.520285 an

In [ ]:
# Now that we have a naive implementation of the softmax loss function and its gradient,
# implement a vectorized version in softmax_loss_vectorized.
# The two versions should compute the same results, but the vectorized version should be
# much faster.
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('naive loss: %e computed in %fs' % (loss_naive, toc - tic))

from cs231n.classifiers.softmax import softmax_loss_vectorized
tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic))

# As we did for the SVM, we use the Frobenius norm to compare the two versions
# of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('Loss difference: %f' % np.abs(loss_naive - loss_vectorized))
print('Gradient difference: %f' % grad_difference)

naive loss: 2.421511e+00 computed in 0.029549s
vectorized loss: 2.421511e+00 computed in 0.028719s
Loss difference: 0.000000
Gradient difference: 0.000000


In [ ]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs231n.classifiers import Softmax
results = {}
best_val = -1
best_softmax = None

# learning_rates = [1e-7, 5e-7]
# regularization_strengths = [2.5e4, 5e4]

learning_rates = [1e-6, 5e-6]
regularization_strengths = [2.5e4, 5e4]


################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################



for l in learning_rates:
    for r in regularization_strengths:
        softmax = Softmax()    
        loss_hist = softmax.train(X_train, y_train, learning_rate=l, reg=r,
                      batch_size=256, num_iters=10000, verbose=True)
        y_train_pred = softmax.predict(X_train)
        training_accuracy = np.mean(y_train == y_train_pred)
        y_val_pred = softmax.predict(X_val)
        validation_accuracy = np.mean(y_val == y_val_pred)
        results[(l, r)] = training_accuracy, validation_accuracy
        if (validation_accuracy > best_val):
            best_val = validation_accuracy
            best_softmax = softmax


################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

iteration 0 / 10000: loss 781.692434
iteration 100 / 10000: loss 768.125484
iteration 200 / 10000: loss 760.857865
iteration 300 / 10000: loss 755.633831
iteration 400 / 10000: loss 750.782640
iteration 500 / 10000: loss 746.403009
iteration 600 / 10000: loss 742.590717
iteration 700 / 10000: loss 739.182557
iteration 800 / 10000: loss 736.055121
iteration 900 / 10000: loss 732.825287
iteration 1000 / 10000: loss 730.200414
iteration 1100 / 10000: loss 727.487346
iteration 1200 / 10000: loss 725.025421
iteration 1300 / 10000: loss 722.635576
iteration 1400 / 10000: loss 720.276226
iteration 1500 / 10000: loss 718.142649
iteration 1600 / 10000: loss 716.231400
iteration 1700 / 10000: loss 714.229272
iteration 1800 / 10000: loss 712.390359
iteration 1900 / 10000: loss 710.657800
iteration 2000 / 10000: loss 708.943657
iteration 2100 / 10000: loss 707.027310
iteration 2200 / 10000: loss 705.660777
iteration 2300 / 10000: loss 704.036885
iteration 2400 / 10000: loss 702.613986
iteration 25

iteration 400 / 10000: loss 706.658931
iteration 500 / 10000: loss 699.813047
iteration 600 / 10000: loss 693.358251
iteration 700 / 10000: loss 687.856487
iteration 800 / 10000: loss 682.781071
iteration 900 / 10000: loss 678.301031
iteration 1000 / 10000: loss 674.327570
iteration 1100 / 10000: loss 670.731525
iteration 1200 / 10000: loss 667.191415
iteration 1300 / 10000: loss 664.189095
iteration 1400 / 10000: loss 661.342167
iteration 1500 / 10000: loss 658.840680
iteration 1600 / 10000: loss 656.429361
iteration 1700 / 10000: loss 654.193778
iteration 1800 / 10000: loss 652.271057
iteration 1900 / 10000: loss 650.273410
iteration 2000 / 10000: loss 648.369032
iteration 2100 / 10000: loss 646.768329
iteration 2200 / 10000: loss 645.140732
iteration 2300 / 10000: loss 643.631900
iteration 2400 / 10000: loss 642.117135
iteration 2500 / 10000: loss 640.704094
iteration 2600 / 10000: loss 639.399376
iteration 2700 / 10000: loss 638.196952
iteration 2800 / 10000: loss 637.184297
iterat

In [ ]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

**Inline Question** - *True or False*

It's possible to add a new datapoint to a training set that would leave the SVM loss unchanged, but this is not the case with the Softmax classifier loss.

*Your answer*:

*Your explanation*:

In [ ]:
# Visualize the learned weights for each class
w = best_softmax.W[:-1,:] # strip out the bias
w = w.reshape(32, 32, 3, 10)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2, 5, i + 1)
    
    # Rescale the weights to be between 0 and 255
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])